In [ ]:
import time
import psutil

import numpy as np
import pandas as pd

import xarray
import dask

from gsat import compute, measure

In [ ]:
dask.config.set(scheduler="processes")

# Performance

In [ ]:
df = pd.read_csv("../../../data_inventory.csv")
subset = df.query('type == "netcdf" & variable == "t" & project == "CMIP6" & frequency == "mon"')
subset

In [ ]:
hist = xarray.open_dataset(subset[subset["experiment"] == "historical"]["location"].iloc[0])
ssp126 = xarray.open_dataset(subset[subset["experiment"] == "ssp126"]["location"].iloc[0])
ssp245 = xarray.open_dataset(subset[subset["experiment"] == "ssp245"]["location"].iloc[0])
ssp370 = xarray.open_dataset(subset[subset["experiment"] == "ssp370"]["location"].iloc[0])
ssp585 = xarray.open_dataset(subset[subset["experiment"] == "ssp585"]["location"].iloc[0])

## Set up experiment

In [ ]:
nworkers = [8, 4, 2, 1]
runs = 5

Run the experiment. Possible names:

- `gpfs` - Run from GPFS and not influenced by NFS.
- `nfs` - Run from Hub and accessing "local" files through NFS.
- `tds-hub` - Run from Hub and accessing OPeNDAP.
- `tds-hub-deflate` - Run from Hub and accessing OPeNDAP (compression enabled).
- `tds-remote` - Rum from remote and accessing OPeNDAP.
- `tds-remote-deflate` - Rum from remote and accessing OPeNDAP (compression enabled).

In [ ]:
results = measure("local", nworkers, runs,
                  hist, ssp126, ssp245, ssp370, ssp585)

Collect results.

In [ ]:
pd.DataFrame.from_records(results).to_csv("hub.csv", index=False)

# Change .dodsrc

In [ ]:
!sed -i 's|DEFLATE=0|DEFLATE=1|' /home/jovyan/.dodsrc

In [ ]:
!cat /home/jovyan/.dodsrc

In [ ]:
!sed -i 's|DEFLATE=1|DEFLATE=0|' /home/jovyan/.dodsrc

In [ ]:
!cat /home/jovyan/.dodsrc